# Cell assemblies

In [1]:
import sys
sys.path.append('../')
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

plt.rcParams["figure.figsize"] = [16,9]
sns.set(font_scale=3.0)

from network import run_network_recall, train_network
from connectivity import designed_matrix_sequences
from analysis import get_recall_duration_for_pattern, get_recall_duration_sequence

In [ ]:
N = 10
tau_z = 0.050
tau_z_post = 0.005
tau_w = 0.020

training_time = 0.100
inter_sequence_time = 0.000
dt = 0.001
sequence1 = [0, 1, 2, 3]
sequence2 = [4, 5, 6, 7, 8, 9]
sequences = [sequence1, sequence2]

dic = train_network(N, dt, training_time, inter_sequence_time, sequences, tau_z,
                    tau_z_post, tau_w, epochs=3, max_w=5.0, min_w=-3.0)

w = dic['w']
x_total = dic['x']
z_history = dic['z']
z_post_history = dic['z_post']

plt.matshow(w, cmap='seismic')
plt.colorbar();